In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%cd ../../

/home/tejomay/cgpos


In [5]:
import random 

import torch
import torch.nn as nn

from src.model import Transformer
import src.config as cfg
from src.util import read_pkl, encode, decode, get_batch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [54]:
# Given data
orig_tokens = ["John", "Johanson", "'s",  "house"]
labels = ["NNP",  "NNP",      "POS", "NN"]

bert_tokens = ["[CLS]", "john", "johan", "##son", "'", "s", "house", "[SEP]"]
orig_to_tok_map = [1, 2, 4, 6]

# Initialize an empty list for token labels
token_labels = []

# Create a mapping from token indices to label indices
label_idx = 0

for i, orig_idx in enumerate(orig_to_tok_map):
    if i == len(orig_to_tok_map) - 1:
        # If it's the last token, just repeat the label for all subtokens.
        token_labels.append(labels[label_idx])
    else:
        # Check if the next original token index is different.
        if orig_to_tok_map[i + 1] != orig_idx:
            # If different, increment the label index.
            label_idx += 1
        # Append the label corresponding to the current original token.
        token_labels.append(labels[label_idx])

# Now, token_labels contains the labels for each token in bert_tokens.
print(token_labels)

['NNP', 'POS', 'NN', 'NN']


In [15]:
ft_syl = read_pkl(cfg.ft_syl)
ft_targets = read_pkl(cfg.ft_targets)
assert len(ft_syl) == len(ft_targets)

In [19]:
ft_targets_map = read_pkl(cfg.ft_targets_map)

In [38]:
list(zip(ft_syl, ft_targets))[:10]

[(['Θου', 'κυ', 'δί', 'δης'], [2, 0, 1, 0, 0, 0, 1, 1, 0]),
 (['Ἀ', 'θη', 'ναῖ', 'ος'], [2, 0, 1, 0, 0, 0, 1, 1, 0]),
 (['ξυ', 'νέ', 'γρα', 'ψε'], [5, 3, 1, 7, 1, 1, 0, 0, 0]),
 (['τὸν'], [1, 0, 1, 0, 0, 0, 1, 4, 0]),
 (['πό', 'λε', 'μον'], [2, 0, 1, 0, 0, 0, 1, 4, 0]),
 (['τῶν'], [1, 0, 2, 0, 0, 0, 1, 2, 0]),
 (['Πε', 'λο', 'πον', 'νη', 'σί', 'ων'], [2, 0, 2, 0, 0, 0, 1, 2, 0]),
 (['καὶ'], [8, 0, 0, 0, 0, 0, 0, 0, 0]),
 (['Ἀ', 'θη', 'ναί', 'ων'], [2, 0, 2, 0, 0, 0, 1, 2, 0]),
 ([','], [12, 0, 0, 0, 0, 0, 0, 0, 0])]

In [4]:
train_size = 0.98
n_chunks = 500
random_seed = 20
unc_rate = 0.01

In [5]:
data = read_pkl(cfg.pt_syl)
vocab = ["<UNK>"] + sorted(set(data))
data = [d if random.random() > unc_rate else "<UNK>" for d in data]
vocab_size = len(vocab)
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}

In [19]:
d = defaultdict(lambda: 7, {4: 6, 7: 8})

In [24]:
d

defaultdict(<function __main__.<lambda>()>, {4: 6, 7: 8, 10: 7})

In [7]:
# Train and test split
tokens = torch.tensor(encode(stoi, data), dtype=torch.long)
chunks = torch.split(tokens, len(data) // (n_chunks - 1))
l = [1] * int(n_chunks * train_size) + [0] * int(n_chunks * (1 - train_size))
random.shuffle(l)
train_data = torch.cat([chunks[i] for i, v in enumerate(l) if v])
val_data = torch.cat([chunks[i] for i, v in enumerate(l) if not v])

In [8]:
x, y = get_batch(train_data, 256, 64, "cpu")

In [44]:
model = Transformer(
    vocab_size=17016, 
    block_size=256, 
    emb_size=512, 
    n_layer=6, 
    n_head=8, 
    dropout=0.6, 
    device=device
)
model.load_state_dict(torch.load('wts/wts_2023-10-08_03-49-37.pth', map_location=torch.device(device)))
# Omg!!
model.lm_head = nn.Linear(512, len(ft_targets_map[1][0]))

m = model.to(device)

In [28]:
model.lm_head

Linear(in_features=512, out_features=16970, bias=True)

In [42]:
class POSTagger(nn.Module):
    